In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model
from keras.layers import merge, Input
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

In [ ]:
from pathlib import Path
train_dir = "kaggle_bee_vs_wasp/"
bs = 64 # Batch size
resize_size = 224 # for training, resize all the images to a square of this size
training_subsample = 0.3 # for development, use a small fraction of the entire dataset rater than full dataset

bees_vs_wasps_dataset_path=Path(train_dir) # this is relative to the "example_notebook" folder. Modify this to reflect your setup
df_labels = pd.read_csv(bees_vs_wasps_dataset_path/'labels.csv')
df_labels=df_labels.set_index('id')
# perform dataset subsampling
df_labels = df_labels.sample(frac=training_subsample, axis=0)
insect_class = {'bee': 0,'wasp': 1, 'insect': 2,'other': 3} 
#df_labels.label = [insect_class[item] for item in df_labels.label] 
df_labels = df_labels[['path','label']]

In [ ]:
#Uncomment to read data from csv
#df_labels = pd.read_csv("Dataset_full.csv") #full dataset
#df_labels = pd.read_csv("Dataset_10precent.csv") #0.1 fraction for development

In [ ]:
train_df, test_df = train_test_split(df_labels, test_size=0.1)
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
image_input = Input(shape=(resize_size, resize_size, 3))
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')
last_layer = model.get_layer('block5_pool')
last_output = last_layer.output
x = Flatten()(last_output)
custom_vgg_model = Model(image_input, x)
custom_vgg_model.compile(loss='sparse_categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])
for layer in custom_vgg_model.layers:
	layer.trainable = False
custom_vgg_model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    train_dir,  
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1,
    shuffle = False
)


filenames = train_generator.filenames
n_train_samples = len(filenames)
train_features = custom_vgg_model.predict(train_generator, steps=n_train_samples, verbose=1)






In [ ]:

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, 
    train_dir, 
    x_col='path',
    y_col='label',
    class_mode='sparse',
    target_size=(resize_size, resize_size),
    batch_size=1,
    shuffle = False
)


filenames = test_generator.filenames
n_test_samples = len(filenames)
test_features = custom_vgg_model.predict(train_generator, steps=n_test_samples, verbose=1)

In [ ]:

#X = pd.read_csv("train_features.csv")
X = train_features
y = np.array(train_generator.labels)

In [ ]:
test_features.shape

In [ ]:
from sklearn import svm
lin_svm = svm.SVC(kernel='linear')
quadratic_svm = svm.SVC(kernel='poly', degree=2)
exp_svm = svm.SVC(kernel='rbf')
lin_svm.fit(X,y)
pickle.dump(lin_svm, open('lin_svm.sav', 'wb'))

In [ ]:
quadratic_svm.fit(X,y)
pickle.dump(quadratic_svm, open('quadratic_svm.sav', 'wb'))


In [ ]:
exp_svm.fit(X,y)
pickle.dump(exp_svm, open('exp_svm.sav', 'wb'))

In [ ]:
from sklearn.metrics import accuracy_score

lin_score =  accuracy_score(test_generator.labels, test_features)
quadratic_score = accuracy_score(test_generator.labels, test_features)
exp_score = accuracy_score(test_generator.labels, test_features)

In [ ]:
print("Lin score " + str(lin_score))
print("Quadratic score " + str(quadratic_score))
print("Exp score " + str(exp_score))

In [ ]:
np.savetxt("model2_svm/lin_score.csv",  
           np.array([lin_score]), 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model2_svm/quadratic_score.csv",  
           np.array([quadratic_score]), 
           delimiter =", ",  
           fmt ='% s') 
np.savetxt("model2_svm/exp_score.csv",  
           np.array([exp_score]), 
           delimiter =", ",  
           fmt ='% s') 